![alt text](./img/header.png)

# SEAWAT Exercise D: Solute and Heat Transport Exercise

This notebook is based on the problem presented in the SEAWAT Version 4 manual.  The problem consists of a two-dimensional cross section of a confined coastal aquifer initially saturated with relatively cold seawater at a temperature of 5 degrees Celsius. Warmer freshwater with a temperature of 25 degrees C is injected into the coastal aquifer along the left boundary to represent flow from inland areas. The warmer freshwater flows to the right, where it discharges into a vertical ocean boundary. The ocean boundary is represented with hydrostatic conditions based on a fluid density calculated from seawater salinities at 5 degrees C. No-flow conditions are assigned to the top and bottom boundaries. A complete list of the input values used for the problem is given in the SEAWAT Version 4 manual. This problem is a simplified representation of what might occur in a coastal carbonate platform.

In [ ]:
# Setup the python environment
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy

## Case 1 -- Salinty and temperature simulation

Case 1 is the simplest of these cases. It is a variable-density simulation in which the fluid density is a function only of salinity. Although temperature is represented as species 2, it is represented as a conservative tracer in which thermal equilibration with the solid as well as heat conduction are not represented.

In [ ]:
# Grid information
lx = 2000.
lz = 1000.

nlay = 25 # 100
nrow = 1
ncol = 50 # 200

delr = lx / ncol
delc = 1.
delz = lz / nlay
top = 0.0
botm = [top - ((i + 1) * delz) for i in range(nlay)]

# Temporal discretization
nper = 1
nstp = 80
perlen = 200000

# Ibound and strt
ibound = np.ones((nlay, nrow, ncol), dtype=np.int)
strt = 0.

# Hydraulic properties
hk = 10.
vk = 0.1
ss = 1.e-5
sy = 0.2

# Fluid properties
denseref = 1000.
drhodc = 25. / 35.
ocean_conc = 35.
conc_rho_ref = 0.
ocean_density = denseref + drhodc * (ocean_conc - conc_rho_ref)

# Inflow to left side
wel_q = 10.0 / nlay
wel_flow_sp = {0:[(k, 0, 0, wel_q) for k in range(nlay)]}

# Create a numpy recarray of the constant-head list
ocean_h = 0.0
chd_ra = flopy.modflow.ModflowChd.get_empty(ncells=nlay, 
                                            aux_names=['chddensopt', 'chdden'])
chd_ra.k = np.arange(nlay)
chd_ra.i = 0
chd_ra.j = ncol - 1
chd_ra.shead = ocean_h
chd_ra.ehead = ocean_h
chd_ra.chddensopt = 1
chd_ra.chdden = ocean_density
chd_flow_sp = {0 : chd_ra}

In [ ]:
# Transport properties
alphal = 1
alphat = 0.1
dm_salinity = 1.e-10

dt0 = 0.
sconc1 = 35.
sconc2 = 5.
porosity = 0.35

# Set up the concentration save times
dtsave = 5000.
timprs = np.arange(dtsave, perlen + dtsave, dtsave)

# Transport boundary for left side
itype = flopy.mt3d.Mt3dSsm.itype_dict() 
wel_conc = 0.0
wel_temp = 25.0
wel_itype = itype['WEL']
wel_conc_sp = {0:[(k, 0, 0, wel_conc, wel_itype, wel_conc, wel_temp) 
                       for k in range(nlay)]}

# Transport boundary for right side
ocean_conc = 35.0
ocean_temp = 5.0
chd_conc_sp = {0:[(k, 0, ncol-1, ocean_conc, itype['CHD'], ocean_conc, ocean_temp)
                  for k in range(nlay)]}

ssm_data = wel_conc_sp
ssm_data[0].extend(chd_conc_sp[0])

In [ ]:
# Build the flopy SEAWAT model
model_ws = os.path.join('work', 'exSEAWAT_D')
if not os.path.isdir(model_ws):
    os.mkdir(model_ws)

m = flopy.seawat.Seawat('d1', model_ws=model_ws)
dis = flopy.modflow.ModflowDis(m, nlay=nlay, nrow=nrow, ncol=ncol,
                               top=top, botm=botm, delr=delr, delc=delc,
                               perlen=perlen, steady=False, nstp=nstp)
bas = flopy.modflow.ModflowBas(m, ibound=ibound, strt=strt)
lpf = flopy.modflow.ModflowLpf(m, hk=hk, vka=vk, ss=ss)
chd = flopy.modflow.ModflowChd(m, stress_period_data=chd_flow_sp, 
                               dtype=chd_ra.dtype,
                               options=['aux chddensopt aux chdden'])
wel = flopy.modflow.ModflowWel(m, stress_period_data=wel_flow_sp)
pcg = flopy.modflow.ModflowPcg(m, hclose=1.e-5, rclose=1.)
oc = flopy.modflow.ModflowOc(m)

vdf = flopy.seawat.SeawatVdf(m, mtdnconc=-1, mfnadvfd=1, nswtcpl=0,
                             iwtable=0, densemin=0.0, densemax=0.0,
                             denseref=denseref, drhodprhd=0.0, prhrhd=0.0,
                             nsrhoeos=1, drhodc=drhodc,
                             crhoref=conc_rho_ref)

btn = flopy.mt3d.Mt3dBtn(m, prsity=porosity, ncomp=2, mcomp=2, sconc=sconc1,
                         sconc2=sconc2, nprs=-1, timprs=timprs)
adv = flopy.mt3d.Mt3dAdv(m, mixelm=-1, percel=1.0,)
gcg = flopy.mt3d.Mt3dGcg(m, mxiter=1, iter1=200, cclose=1.e-6, isolve=2)
dsp = flopy.mt3d.Mt3dDsp(m, al=alphal, trpt=alphat, trpv=alphat,
                         dmcoef=dm_salinity, dmcoef2=0.)
ssm = flopy.mt3d.Mt3dSsm(m, stress_period_data=ssm_data)

In [ ]:
# Make plot of the grid
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
linecollection = mm.plot_grid()
patchcollection = mm.plot_bc(ftype='wel')
patchcollection = mm.plot_bc(ftype='chd')

In [ ]:
m.write_input()

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
try:
    os.remove(fname)
except:
    pass
m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Salinity Concentration at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Extract temperature
fname = os.path.join(model_ws, 'MT3D002.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Temperature at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=5, vmax=25, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

## Case 2 -- Add effects of temperature on density

The effect of temperature on fluid density is activated for Case 2.

In [ ]:
m.name = 'd2'
drhodt = -0.375
temp_rho_ref = 25.

ocean_density = denseref + drhodc * (ocean_conc - conc_rho_ref) + \
                           drhodt * (ocean_temp - temp_rho_ref)
ocean_h = 0.0
chd_ra = flopy.modflow.ModflowChd.get_empty(ncells=nlay, 
                                            aux_names=['chddensopt', 'chdden'])
chd_ra.k = np.arange(nlay)
chd_ra.i = 0
chd_ra.j = ncol - 1
chd_ra.shead = ocean_h
chd_ra.ehead = ocean_h
chd_ra.chddensopt = 1
chd_ra.chdden = ocean_density
chd_flow_sp = {0 : chd_ra}

chd = flopy.modflow.ModflowChd(m, stress_period_data=chd_flow_sp, 
                               dtype=chd_ra.dtype,
                               options=['aux chddensopt aux chdden'])

vdf = flopy.seawat.SeawatVdf(m, mtdnconc=-1, mfnadvfd=1, nswtcpl=0,
                             iwtable=0, densemin=0.0, densemax=0.0,
                             denseref=denseref, drhodprhd=0.0, prhrhd=0.0,
                             nsrhoeos=2, drhodc=[drhodc, drhodt],
                             crhoref=[conc_rho_ref, temp_rho_ref])

In [ ]:
m.write_input()

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
try:
    os.remove(fname)
except:
    pass
m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Salinity Concentration at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

## Case 3 -- Heat Conduction

In Case 3, heat conduction is included in the simulation by specifying a value for bulk thermal diffusivity ($D_{m\_temp}$) within the DSP input file. A bulk thermal diffusivity value of 0.15 $m^2/d$ ($1.74 × 10^{-6} m^2/s$) was calculated using equation 25 of the SEAWAT manual.

In [ ]:
m.name = 'd3'
ktbulk = 2.547
cpfluid = 4183.
dm_temperature = ktbulk / porosity / denseref / cpfluid
print('dm_temperature = {} m2/d'.format(dm_temperature))
dm_temperature = dm_temperature * 86400 
print('dm_temperature = {} m2/s'.format(dm_temperature))

In [ ]:
dsp = flopy.mt3d.Mt3dDsp(m, al=alphal, trpt=alphat, trpv=alphat,
                         multiDiff=True, dmcoef=dm_salinity, 
                         dmcoef2=dm_temperature)

In [ ]:
m.write_input()

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
try:
    os.remove(fname)
except:
    pass
m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Salinity Concentration at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Extract temperature
fname = os.path.join(model_ws, 'MT3D002.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Temperature at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=5, vmax=25, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

## Case 4 -- Thermal equilibration with solid

In Case 4, thermal equilibration between the fluid and the solid matrix is included in the simulation. This feature is activated using the MT3DMS Reactions (RCT) Package, which will be created and added to the name file. This "reaction" process is activated only for the temperature species (species 2) by entering $2.0 × 10^{-4} m^3 /kg$ for Kd_temp. 

In [ ]:
m.name = 'd4'
rhobulk = 1761.5
kd_temp = 2.e-4
kd_salinity = 0.

In [ ]:
rct = flopy.mt3d.Mt3dRct(m, isothm=1, igetsc=0, rhob=rhobulk, sp1=kd_salinity,
                         sp12=kd_temp)

In [ ]:
m.write_input()

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
try:
    os.remove(fname)
except:
    pass
m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Salinity Concentration at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Extract temperature
fname = os.path.join(model_ws, 'MT3D002.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Temperature at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=5, vmax=25, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

## Case 5 -- Assigning a constant temperature boundary to the ocean

Heat conduction at the seawater boundary is included in the Case 5 simulation by changing the MT3DMS boundary type for the temperature species to ITYPE = -1. By using the constant-head boundary type (ITYPE = 1 in the SSM input file), only advective transport (no dispersive transport) is allowed across the boundary. With this condition, the solute concentration (or temperature) value at the constant-head boundary can change from the value specified in the SSM input file. Without a dispersive flux across the boundary, there is no heat conduction. By setting ITYPE = -1 for temperature, the assigned temperature value will remain constant at the boundary, and heat conduction to or from the boundary will occur.  

In [ ]:
m.name = 'd5'

# leave the well the same as it was
itype = flopy.mt3d.Mt3dSsm.itype_dict() 
wel_conc = 0.0
wel_temp = 25.0
wel_itype = itype['WEL']
wel_conc_sp = {0:[(k, 0, 0, wel_conc, wel_itype, wel_conc, wel_temp) 
                       for k in range(nlay)]}

ocean_conc = 35.0
ocean_temp = 5.0
chd_conc_sp = [(k, 0, ncol - 1, ocean_conc, itype['CHD'], ocean_conc, -1)
                  for k in range(nlay)]
chd_conc_sp += [(k, 0, ncol - 1, ocean_conc, itype['CC'], -1, ocean_temp)
                  for k in range(nlay)]

ssm_data = wel_conc_sp
ssm_data[0].extend(chd_conc_sp)

In [ ]:
ssm = flopy.mt3d.Mt3dSsm(m, stress_period_data=ssm_data)

In [ ]:
m.write_input()

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
try:
    os.remove(fname)
except:
    pass
m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Salinity Concentration at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Extract temperature
fname = os.path.join(model_ws, 'MT3D002.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Temperature at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=5, vmax=25, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

If you get the chance, take a look at the ssm file, which is called d5.ssm.  The first line indicates which flow packages are active. The “T” indicates that the well package is active. The second line specifies the maximum number of sources and sinks present in the simulation. The third line indicates the number of sources or sinks that are specified in the subsequent data record. The subsequent lines contain layer, row, column, a dummy value, ITYPE, and the concentration value for each species. A negative concentration value indicates that this boundary type does not apply for this species. The first set of lines specify the salinity and temperature values for the inflow along the left boundary, which is specifies using the well package. The second set of lines indicate the right boundary has a zero dispersive flux (ITYPE = 1) for salinity (species 1) and that any solute entering from the boundary will have a concentration of 35 kg/m3. The last set of lines show ITYPE = -1 to indicate that temperature will be held constant at 5 degrees C. 

You'll note that for this particular problem, the use of a constant temperature boundary does not have a substantial effect on the results as shown in the temperature plot above. Simulated salinities for Case 5 are the same as those for Case 4. Including the heat conduction at the boundary affects simulated temperatures slightly, but only near the vertical ocean boundary.

## Case 6 -- Viscosity effects

For Case 6, the effects of fluid viscosity variation on salinity and temperature have been included in the simulation. Viscosity effects are included by activating the VSC Package in the name file.

In [ ]:
m.name = 'd6'

In [ ]:
vsc = flopy.seawat.SeawatVsc(m, mt3dmuflg=-1, viscmin=0, viscmax=0, nsmueos=1,
                             viscref=8.904e-4, mutempopt=1, mtmuspec=1,
                             dmudc=1.923e-6, cmuref=0.0, mtmutempspec=2,
                             amucoeff=[239.4e-7, 10.0, 248.37, 133.15])

In [ ]:
m.write_input()

In [ ]:
fname = os.path.join(model_ws, 'MT3D001.UCN')
try:
    os.remove(fname)
except:
    pass
m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Salinity Concentration at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Extract temperature
fname = os.path.join(model_ws, 'MT3D002.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])

# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.set_title('Temperature at time = {} days'.format(times[-1]))
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
cpatchcollection = mm.plot_array(conc, vmin=5, vmax=25, edgecolor='k')
cb = plt.colorbar(cpatchcollection)

If you get the chance, take a look at d6.vsc.  The first line is a comment. The second line indicates that viscosity is a function of one or more MT3DMS species. The third line contains the viscosity limiters, which are not active because of the zero value. The fourth line is the reference viscosity. The fifth line contains two important flags. The first indicates that one of the species has a linear effect on fluid density, and that information for the species will be read on the following line. The second flag contains information about the effects of temperature on fluid viscosity. A value of 1 indicates that fluid viscosity is calculated using equation 18 of the SEAWAT manual, and that additional information regarding the temperature/viscosity relation will be read later. The sixth line contains the MT3DMS species number that has a linear effect on fluid density, the slope of this relation, and the reference concentration this relation is based upon. The last line indicates that species 2 is the temperature species and provides constants used in equation 18.

Results from the Case 6 simulation are shown above, and suggest that viscosity variations have a minimal effect on simulated salinities and temperatures for this particular problem. There are some minor differences between Cases 5 and 6, particularly at later times, but the overall results are similar.